# NLE Assessed Coursework 2: Question 1

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 1 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [9]:
candidateno=181345 #this MUST be updated to your candidate number so that you get a unique data sample


In [10]:
#preliminary imports
import sys
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'/Users/juliewe/Documents/teaching/NLE2018/resources')
sys.path.append(r'/Users/lucaskoh/Documents/NLE/resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from nltk.tokenize import word_tokenize

from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
import random
from nltk.corpus import stopwords
import spacy
nlp=spacy.load('en')
from nltk.corpus import gutenberg

## Question 1: Part-of-Speech Tagging (25 marks)


In [11]:
#Do NOT change the code in this cell.

#preparing corpus
def display_sent(asent):
    headings=["token","lower","lemma","pos","NER"]
    info=[]
    for t in asent:
        info.append([t.text,t.lower_,t.lemma_,t.pos_,t.ent_type_])
    return(pd.DataFrame(info,columns=headings))

def tag_sent(asent):
    tagged=[]
    for t in asent:
        tagged.append((t.lower_,t.pos_))
    return tagged

def clean_text(astring):
    #replace newlines with space
    newstring=re.sub("\n"," ",astring)
    #remove title and chapter headings
    newstring=re.sub("\[[^\]]*\]"," ",newstring)
    newstring=re.sub("VOLUME \S+"," ",newstring)
    newstring=re.sub("CHAPTER \S+"," ",newstring)
    newstring=re.sub("\s\s+"," ",newstring)
    #return re.sub("([^\.|^ ])  +",r"\1 .  ",newstring).lstrip().rstrip()
    return newstring.lstrip().rstrip()

def display_tags(sentslist):
    taglist={}
    for s in sentslist:
        for t in s:
            form=t.lower_
            pos=t.pos_
            taglist[pos]=taglist.get(pos,0)+1
    print(len(taglist.keys()))
    print(taglist)
        
def get_train_test(sentslist,seed=candidateno):
    random.seed(seed)
    random.shuffle(sentslist)
    testsize=10
    train=[tag_sent(s) for s in sentslist[testsize:]]
    test=[tag_sent(s) for s in sentslist[:testsize]]
    return train,test
    
alice=clean_text(gutenberg.raw('carroll-alice.txt'))
nlp_alice=list(nlp(alice).sents)

The code below will generate (unique to you) training and test sets of pos-tagged sentences from the novel "Alice in Wonderland"

In [12]:
#do not change the code in this cell
allsents=list(nlp_alice)
train,test=get_train_test(allsents)

In the cell below, a class `unigram_tagger` is defined, which will be used in parts a and b of this question

In [13]:
#do not change the code in this cell
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
                

a) Create an instance of the `unigram_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).
Explain what is stored, after training, in the three instance variables:
* `self.tagcounts`
* `self.wordcounts`
* `self.tagperwordcounts`

You should refer to the code and specific examples from the training data. \[4 marks \]

In [14]:
alice_train = unigram_tagger(train)

Once the train data is passed through the instance of a `unigram_tagger` class: For each token within the training corpus, it will assign the tag that is most likely for that token's type. 

The **self.tagcounts** variable stores a dictionary of the invidual types of `parts of speech` occuring within the training corpus following by its count. The key is represented by the parts of speech tag and the value represents its occurence within the training data.For instance, when looking through the dictionary, there are a total of *2859*`determiners` present within the training data compared to the quantity of `verbs` being *6210*.
The parts of speech is useful when performing **Word-sense disambiguation (WSD)** :identifying which sense of a word, the meaning used in a sentence, when the word has multiple meanings. For instance, given the word `live` from the training data, without the parts of speech tag, it is difficult to determine the meaning of the word. Evidently, the context of of the word would help, however there may be multiple interpretations of the same word.

The **self.wordcounts** variable stores a dictionary of the individual words from the corpus followed by its occurence. The key is is comprised of the individual words within the training data and the value represents its corresponding occurences. For instance, the word `for` is present *152* times whereas `tortoise` is present *2* times within the training data. Hence, the variable displays how `often a word occured` within the corpus. 

The **self.tagperwordcounts** variable is a nested dictionary comprised of a key representing each individual word from the training data, and a value represented by a dictionary of the word's possible `parts of speech` tags as the nested dictionary's key and the occurences of the each tag as the value. For intance, the word `it` is tagged and used as a `pronoun` *585* times, as a `noun` *3* times and a `proper noun` *1* time. it': 
{'PRON': 585, 'NOUN': 3, 'PROPN': 1}

b) In the cells below, **extend** the code for the `unigram_tagger` class so that it also has a `tag()` method.  This method should assign the tag, $t$, which maximises the unigram tag probability, $P(t|w)$, for the observed word, $w$.  **Evaluate** the performance of the `unigram_tagger` on your test sentences.  **Discuss** your results. \[8 marks\] 



In [15]:
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        self.train_tagged={}
        self.tagged={}
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
    
    #This method will tag the word within a given corpus depending on
    #maximum value of its tag count - most likely
    def tag(self,data):
        for word,tags in self.tagperwordcounts.items():
            for indi_tag in tags: 
                self.train_tagged[word] = max(tags,key=tags.get)
            for sentence in data:
                for word in sentence:
                    self.tagged[word[0]] = self.train_tagged.get(word[0])

In [8]:
new_train = unigram_tagger(train)
new_train.tag(test)
sum = 0
correct_tags = 0
for sentence in test: 
    for word in sentence: 
        sum = sum + 1 
        for tag_word,tag in new_train.tagged.items():
            if word[0] == tag_word and word[1] == tag: 
                correct_tags = correct_tags + 1
accuracy = ((100/sum)*correct_tags)
print("The accuracy of the tagger is ",accuracy,"%")

The accuracy of the tagger is  92.5531914893617 %


`The unigram tagger has an accuracy is approximately 93%`. The probability is estimated using training data and is represented as such: 

$$p(t|w) = \frac{number \ of \ occurences \ of \ w \ tagged \ as\  t}{number \ of\  occurences \ of\  w}$$


**Accuracy** represents a standard way of measuring the performance of a parts-of-speech tagger. Hence, the unigram tagger's accuracy represents the percentage of tags correctly labeled when matching human lavels on a test set. A unigram tagger would have significantly higher accuracy compared to a default tagger.

The overall construction of a unigram tagger is as follows: The tagger acquires knowledge of each word token and finds its most frequently appearing tag from the training data `train`. For instance, when training and testing the unigram tagger with the same sentene containing **'the/determiner'** , **'mind/verb'** and **of/adposition**, the tagger will learn to give `'the'` a tag of a *determiner*, `'mind`' a tag of a *verb* and `'of'` a tag of an *adposition*. The reason being is *'determiner'* is the most frequent/likely tag to the word `'the'`, and *'adposition'* is the most frequent tag to the word `'of'` in the `training` sentence.

When performing Parts-of-speech(POS) tagging, the unigram will attempt to classify the words in a sentence aoccrding to their most likely types. In this case of tagging, the unigram tagger will considfer only the current token in isolation from any larger context. For instance, given such model, the tagger would tag the word **('shoulders', 'NOUN')** with the same tag regardless of whether it appears in the context of the `noun` shoulders or the `verb` shoulders. 


In the cell below, a `hmm_tagger` class is defined, which will be used in parts c and d of this question.

In [60]:
#do not change the code in this cell
class hmm_tagger():
    
    def __init__(self,trainingdata=[]):
        
        self.emissions={}
        self.transitions={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            previous="START"
            for token,tag in sentence:
                
                current=self.emissions.get(tag,{})
                current[token]=current.get(token,0)+1
                self.emissions[tag]=current
                bigram=self.transitions.get(previous,{})
                bigram[tag]=bigram.get(tag,0)+1
                self.transitions[previous]=bigram
                previous=tag
                
        self.emissions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.emissions.items()}
        self.transitions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.transitions.items()}

c) **Create** an instance of the `hmm_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).  With reference to your testing sentences (stored in `test`), **explain**
* how to calculate the probability of an observed sequence of words for a given sequence of tags: $$P(w_1^n|t_1^n)$$
* how to calculate the probablity of a possible sequence of tags for a given sequence of words: $$P(t_1^n|w_1^n)$$

\[6 marks\]

In [67]:
hmm_train = hmm_tagger(train)

### To calculate the probability of an observed sequence of words for a given sequence of tags : 
#### $$P(w_1^n|t_1^n)$$
We will use the **forward algorithm** which calculates the probability of a word sequence given a tag sequence: $$P(w_1^n|t_1^n) = \prod_{i}^{n}P(w_i|t_i)$$

In this case, the test word sequence is (shouted the queen.') given the tag sequence (VERB,DET,PROPN,PUNCT,PUNCT).
In order to calculate this probability, we will need to consider the `emission` probablities. 

The `emission` probability constitutes as one of the paramters of an HMM. It represents the probability of a word given a tag for each **word** *w* and **tag** *t*. For instance, given the test sentence above, the emission probability would represent the probability that we observe the *word* **shouted** given the *tag* is a **verb** and continued for each word. In addition, we also assume that the current word only depends on the current tag. 

Furthermore, in order to obtain the probability of each word within the word sequence given its tag, the `hmm_train.emissions.get()` function was utilized as such:

* 1. hmm_train.emissions.get('VERB').get('shouted'): $$0.0012882447665056361$$

* 2. hmm_train.emissions.get('DET').get('the') : $$0.5647305808257522$$

* 3. hmm_train.emissions.get('PROPN').get('queen') :  $$0.05153901216893343$$

* 4. hmm_train.emissions.get('PUNCT').get('.') :  $$0.13466368964554842$$

* 5. hmm_train.emissions.get('PUNCT').get("'") :  $$0.3060284677644432$$


Therefore, in order to calcualte this probability given: 
$$P(shouted\ the \ queen. '\ | \ VERB\ DET\ PROPN\ PUNCT\ PUNCT)$$

We will take the product of: 

$$P(shouted \ |\ VERB ) \ * \ P(the \ |\ DET )\ * \ P(queen \ |\ PROPN) \ *\  P(. \ |\ PUNCT ) \ * \ P(' \ |\ PUNCT )$$

This is equivalent to: 
$$0.0012882447665056361 * 0.5647305808257522 * 0.05153901216893343 * 0.13466368964554842 * 0.3060284677644432$$

Hence, the probability of an observed sequence of words for a given sequence of tags with reference to one of the test setence is: $$1.5452121716390934e^{-06}$$

### To calculate the probablity of a possible sequence of tags for a given sequence of words: 
#### $$P(t_1^n|w_1^n)$$

We must **maximize** the *tag* sequence given the *word* sequence.
$$\hat{t}_{1}^{n} = \underset{\hat{t}_{1}^{n}}{argmax} {P(t_1^n|w_1^n)}$$

To do so, we will apply Bayes' Rule and calculate the probability of the word sequence given the tag sequence and, multiple the probability of a tag sequence and divide the probability of the word sequence:

$$\hat{t}_{1}^{n} = \underset{t_{1}^{n}}{argmax} \frac{{P(w_1^n|t_1^n)} * {P(t_1^n)}}{P(w_1^n)}$$

Furthermore, as in Naive Bayes Classifiers, the denominator can be dropped. This means that the most likely tag sequence is the one that maximizes the **product** of the probility of the word sequence given the tag sequence times the probability of the tag sequence: 

$$\hat{t}_{1}^{n} = \underset{t_{1}^{n}}{argmax}  {{P(w_1^n|t_1^n)} {P(t_1^n)}}$$

### Two assumptions will be made:
Firstly, we will assume output independence which means that the current observation depends only on the current state. 

$$\hat{t}_{1}^{n} = \underset{t_{1}^{n}}{argmax}  {{P(w_i|t_i)} {P(t_1^n)}}$$

Hence, instead of determining the probabilities of the word sequence given the tag sequence and considering all possible dependencies. This will be equivalent to the product of the probability of each word given each tag. 

Secondly, we will make the first order Markov assumption that : *the current state depends on the single previous state*. 

$$\hat{t}_{1}^{n} = \underset{t_{1}^{n}}{argmax}  {{P(w_i|t_i)} {P(t_1|t_{i-1})}}$$

${P(w_i|t_i)}$ : represents the emission probability which is the probability of a word given a tag for each word w and tag t.

${P(t_1|t_{i-1})}$ : represents the transition probability which is the probability of a tag given some other tag having occured previously.

### Decoding
Now, given a tag sequence and a word sequence, we are able to determine the probability that the word word sequence was generated by that tag sequence. 
$$P(t_1^n|w_1^n) = \prod_{i}^{n}P(w_i|t_i)P(t_i|t_{i-1})$$

Therefore, given two possible tag sequences we are able to choose the one with the highest probability. 

### Problem
As the given test setence used only has one tag sequence **(VERB,DET,PROPN,PUNCT,PUNCT)**, we will carry out the calculation with this in constraint. 

### Calculation
The calculation of the probability of a possible tag sequence given a sequence of words can be devised in to 3 different steps: 

For : $P(VERB\ DET \ PROPN\ PUNCT\ PUNCT \ | \ shouted\ the\  queen\ .\ ')$

1. We will calculate $P(shouted\ the \ queen. '\ | \ VERB\ DET\ PROPN\ PUNCT\ PUNCT)$ using the **forward algorithm**. This would give us, $1.5452121716390934e^{-06}$.

2. We will calculate the transition probabilities $P(VERB\ DET \ PROPN\ PUNCT\ PUNCT)$.
    * Probability of a verb given the start sequence : 'VERB': 0.12433392539964476
    * Probability of a Determiner given previously a verb was seen : 'DET': 0.14398581102870042
    * Probability of a Proper noun given the previous tag being a determiner :  'PRON': 0.004201680672268907
    * Probability of a punctuation given the previous tag being a proper noun : 'PUNCT': 0.13686455741201564
    * Probability of a punction given the previous tag being a punctuation : 'PUNCT': 0.3944805194805195
    Once we have acquired each transition probabilities, we will take the product: 
    $0.12433392539964476 * 0.14398581102870042 * 0.004201680672268907 * 0.13686455741201564 * 0.3944805194805195 = 4.0611491889332635e^{-06}$

3. Now, we will take the product of the values obtained from Step 1 and Step 2 as such : 
$1.5452121716390934e^{-06} * 0611491889332635e^{-06}$

This results: 

$P(VERB\ DET \ PROPN\ PUNCT\ PUNCT \ | \ shouted\ the\  queen\ .\ ')$ = $6.275337157581911e^{-12}$

As mentioned above, as the test setence given only have one tag sequence, we are not able to compare with a second tag sequence to choose which has the highest probability. 


d) Using one of your `test` sentences as an example, **explain** how the Viterbi algorithm can be used to find the most likely tag sequence.  You do not need to write code for this question but should explain the calculations that need to be made at each step.  **Comment** on whether the sequence found by the Viterbi algorithm is correct for your chosen test sentence.  **Discuss** why using the Viterbi algorithm is better than the brute-force approach of enumerating and testing all tag sequence possibilities.\[7 marks\]


## Explain how the viterbi algorithm can be used to find the most likely tag sequence.
The viterbri algorithm can be used to find the most likely tag sequence by recursively decomposing the problem into smaller problems whilst keeping track of the solutions to the sub-problems. 
The algorithm exploits the HMM assumptions that: 
* the probability of next state only depends on current state
* probability of current output only depends on current state 

### How does it work? what are the sub-problems
?
For a given word sequence $W_{1}^{n}$, a sub-problem corresponds to a pair (i,t) where:

* i is the position in the sequence so far, i<n.
* t is the current PoS tag. 

We will be tabulating a solution for the best possible sequence of tags from the start of the sequence to position i where the final tag is t. 

For our given input sequence (test sentence) : **(shouted the queen. ')** the given tag sequence corresponds to :
$(VERB\ DET \ PROPN\ PUNCT\ PUNCT)$
To ease the calculation, we will remove the last two PUNCT tags as it does not directly affect the main meaning of the sentence.
The sub-problems correspond to: 
* when i = 1 `shouted`, we will store the best path when $t_1 = VERB$, $t_1 = DET$, $t_1 = PROPN$
* when i = 2, `the` we will store the best path when $t_2 = VERB$, $t_2 = DET$, $t_2 = PROPN$
* when i = 3, `queen` we will store the best path when $t_3 = VERB$, $t_3 = DET$, $t_3 = PROPN$

The algorithm starts with the initialisation for when i = 1. We are given 3 sub-problems to store the best path for when $t_1 = VERB$, $t_1 = DET$, $t_1 = PROPN$. 
#### sub-problem 1 : $t_1 = VERB$
We will multiple the probability of `shouted` given Verb and the probability of the first tag being a verb.
V(1,VERB) = $P(t_{1} = VERB) = P(shouted|VERB) * P(V|start)$

0.0012882447665056361 * 0.12433392539964476 = 0.00016017252869519454, hence this gives us a probability of **0.00016017252869519454**.

#### sub-problem 2: $t_1 = DET$
We will calculate the path of $t_1$ being $DET$. This is achieved by multiplying the probability of `shouted` given DET with the probability of the first tag being DET. 
V(1,DET) = $P(t_{1} = DET) = P(shouted|DET) * P(DET|start)$

1 * 0.10657193605683836 = 0.10657193605683836, hence the probability is **0.10657193605683836**.

#### sub-problem 3:  $t_1 = PROPN$
We will calculate the path of $t_1$ being $PROPN$. This is achieved by multiplying the probability of `shouted` given PROPN with the probability of the first tag being PROPN. 
V(1,PROPN) = $P(t_{1} = PROPN) = P(shouted|PROPN) * P(PROPN|start)$

1 * 0.07460035523978685 = 0.07460035523978685, hence the probability is **0.07460035523978685**. 

This concludes the **initialisation step**. Now we must apply the **recursive step** at each position of the sequence.

For each subsequent position $i \ \varepsilon \ {(2,...,n)}$ and each tag *t* we will calculate *v(i,t)*

$$V(i,t) = \underset{t' \epsilon T}{max}(V(i-1,t')*P(t|t')*P(w_{i})|t))$$

We will take the maximum over each possible previous tag for $(V(i-1,t')$, which is each possible tag and the previous position of the sentence. Multiple it by the probability of transitioning from that tag to our current tag and with the probability of the current tag generating the word at position *i*. 

### Using the context of the test setnence `shouted the queen`, we will move to when i = 2. 

### sub-problem 1: V(2,VERB)
our first sub-problem consists of calculating the best path where the second tag is tagged as a verb. There are 3 ways of achieving this: 
you could consider the first tag being: 
* a verb
* a determiner
* a proper noun
We must consider each of these possibilities: 

The previous tag could have been a verb, a determiner or a proper noun.

#### * if it was a VERB:

$P(t_2 = VERB) = V(1,VERB) * P(VERB|VERB) * P(the|VERB)$
$P(t_2 = VERB) = 0.00016017252869519454 * 0.12786198000644952 * 0 = 0$

#### * if it was a DET:

$P(t_2 = VERB) = V(1,DET) * P(VERB|DET) * P(the|VERB)$
$P(t_2 = VERB) = 0.10657193605683836 * 0.0430672268907563 * 0 = 0$

#### * if it was a PROPN:

$P(t_2 = VERB) = V(1,PROPN) * P(VERB|PROPN) * P(the|VERB)$
$P(t_2 = VERB) = 0.07460035523978685 * 0.27220216606498193 * 0 = 0$

All the probabilities equate to 0 as the probability of the the word in position 2 being a VERB is 0. 

Furthermore, we must consider the other possible tag for position 2 which is a determiner. 

### sub-problem 2: V(2,DET)

#### * if it was a VERB:

$P(t_2 = DET) = V(1,VERB) * P(DET|VERB) * P(the|DET)$
$P(t_2 = DET) = 0.00016017252869519454 * 0.14398581102870042 * 0.5647305808257522 = 1.302413936955715e^{-05}$

#### * if it was a DET:

$P(t_2 = DET) = V(1,DET) * P(DET|DET) * P(the|DET)$
$P(t_2 = DET) = 0.10657193605683836 * 0.0024509803921568627 * 0.5647305808257522 = 0.00014751086114976288$

#### * if it was a PROPN:

$P(t_2 = DET) = V(1,PROPN) * P(DET|PROPN) * P(the|DET)$
$P(t_2 = DET) = 0.07460035523978685 * 0.010108303249097473 * 0.5647305808257522 = 0.00042585373806585693$

The third probability is the greatest. Therefore, if the second tag is determiner, the first tag must have been a proper noun. **However, we will not make this decision now until we have reached the end of the tag sequence**

### sub-problem 3: V(2,PROPN)

#### * if it was a VERB:

$P(t_2 = DET) = V(1,VERB) * P(PROPN|VERB) * P(the|PROPN)$
$P(t_2 = DET) = 0.00016017252869519454 * 0.032086423734279265 * 0.0007158196134574087 = 3.6788572843987233e^{-09}$

#### * if it was a DET:

$P(t_2 = DET) = V(1,DET) * P(PROPN|DET) * P(the|PROPN)$
$P(t_2 = DET) = 0.10657193605683836 * 0.21358543417366946 * 0.0007158196134574087 = 1.62936386781878e^{-05}$

#### * if it was a PROPN:

$P(t_2 = DET) = V(1,PROPN) * P(PROPN|PROPN) * P(the|PROPN)$
$P(t_2 = DET) = 0.07460035523978685 * 0.09963898916967509 * 0.0007158196134574087 = 5.320761623329303e^{-06}$

Here, the second probability is the greatest. 

Using the same methodology, we will apply this to the last word in the sentence `queen` at position i =3. Thus comparing which probability is the greatest to determine the most likely tag sequence.

### Given `shouted the queen`, we will move to when i = 3. 

### sub-problem 1: V(3,VERB)

#### * if it was a VERB:

$P(t_2 = VERB) = V(1,VERB) * P(VERB|VERB) * P(queen|VERB)$
$P(t_2 = VERB) = 0.00016017252869519454 * 0.12786198000644952 * 0 = 0$

#### * if it was a DET:

$P(t_2 = VERB) = V(1,DET) * P(VERB|DET) * P(queen|VERB)$
$P(t_2 = VERB) = 0.10657193605683836 * 0.0430672268907563 * 0 = 0$

#### * if it was a PROPN:

$P(t_2 = VERB) = V(1,PROPN) * P(VERB|PROPN) * P(queen|VERB)$
$P(t_2 = VERB) = 0.07460035523978685 * 0.27220216606498193 * 0 = 0$


### sub-problem 2: V(3,DET)

#### * if it was a VERB:

$P(t_2 = DET) = V(1,VERB) * P(DET|VERB) * P(queen|DET)$
$P(t_2 = DET) = 0.00016017252869519454 * 0.14398581102870042 * 0 = 0$

#### * if it was a DET:

$P(t_2 = DET) = V(1,DET) * P(DET|DET) * P(queen|DET)$
$P(t_2 = DET) = 0.10657193605683836 * 0.0024509803921568627 * 0 = 0$

#### * if it was a PROPN:

$P(t_2 = DET) = V(1,PROPN) * P(DET|PROPN) * P(queen|DET)$
$P(t_2 = DET) = 0.07460035523978685 * 0.010108303249097473 * 0 = 0$

### sub-problem 2: V(3,PROPN)

#### * if it was a VERB:

$P(t_2 = PROPN) = V(1,VERB) * P(DET|VERB) * P(queen|PROPN)$
$P(t_2 = PROPN) = 0.00016017252869519454 * 0.14398581102870042 * 0 = 0$

#### * if it was a DET:

$P(t_2 = PROPN) = V(1,DET) * P(DET|DET) * P(queen|PROPN)$
$P(t_2 = PROPN) = 0.10657193605683836 * 0.0024509803921568627 * 0 = 0$

#### * if it was a PROPN:

$P(t_2 = PROPN) = V(1,PROPN) * P(DET|PROPN) * P(queen|PROPN)$
$P(t_2 = PROPN) = 0.07460035523978685 * 0.010108303249097473 * 0.05153901216893343 = 3.88646935964923e^{-05}$

Once we have worked out all the probabilities, we can conclude that V(3,PROPN) is the greastest. Therefore, this would be the most likely path where the first tag being a determiner, the second tag being a verb and final tag being a proper noun. [DET,VERB,PROPN].

## Correctness 
The sequence found by the Viterbi algorithm for the chosen test sentence was incorrect. The given sequence [VERB,DET,PROPN] did not correspond to the found sequence [DET,VERB,PROPN]. *This may had to do with the fact that the PUNCT tags were omitted from the intial tag serquence. Furthermore, with the nature of the algorithm using the tags existing solely from the tag sequence may have an affect on the overall likelihood of a specific tag sequence forming*

## Discuss 
When finding the most likely tag sequence of a given sentence, the possible number of tag sequences is represented by 
$k^n$ where `k` represents the |tagset| and `n` represents the |word tokens|. 
This may seem viable for a relatviely small scale problem: a tagset of size 2 and sentence of length 3 is $2^3 = 8$ possible tag sequences. However, if we consider a tagset size of 45 and a sentence length 15, using a brute force method would seem inefficient and impractical.

### Solution
Compared to the Brute force method, the number of sub-problems to consider is `k*n` as given `n` tokens there are `k` sub-problems. In addition, for each sub-problem we must consider `k` previous sub-problems. Hence, the somplexity of the Viterbi algorithm is $k^2*n$
When implementing this on a small scale problem such as a tagset of size 2 and sentence of length 3, the viterbi algorithm would actually perform poorly against the brute force method as it would return 12 possible tag sequences. 
$$12>8$$
However, when applying the Viterbi algorithm to larger scale problems where a tagset size is 45 and a sentence has length 15, the Viterbri algorithm outperforms the Brute force method.
$$30375<<???$$
Ultimately, the Viterbi algorithm will find the most likely tag sequence for a given input word sequence quicker and more efficienctly compared to the Brute force method due to its complexity. 